In [ ]:
"""
Script for plotting results of the ROI-wise Bayesian ANOVAs
"""
import numpy as np
import nibabel as nib
from nilearn import plotting, datasets, image
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap

# path to images
basepath = '/path'

# load standard brain for background
standard_brain = datasets.load_mni152_template()

# define Bayes factors categories and recode function
def recode_bayes_factors(bf_image):
    data = bf_image.get_fdata()
    recoded_data = np.full(data.shape, np.nan)
    
    # define evidence categories
    categories = [
        (-np.inf, -4.61, 1),    # Extreme evidence for H0
        (-4.61, -3.40, 2),      # Very strong evidence for H0
        (-3.40, -2.30, 3),      # Strong evidence for H0
        (-2.30, -1.10, 4),      # Moderate evidence for H0
        (-1.10, 0, 5),          # Anecdotal evidence for H0 now colored
        (0, 1.10, 6),           # Anecdotal evidence for H1 now colored
        (1.10, 2.30, 7),        # Moderate evidence for H1
        (2.30, 3.40, 8),        # Strong evidence for H1
        (3.40, 4.61, 9),        # Very strong evidence for H1
        (4.61, np.inf, 10)      # Extreme evidence for H1
    ]
    
    # recode based on defined ranges
    for (low, high, value) in categories:
        mask = (data > low) & (data <= high)
        recoded_data[mask] = value
    
    return nib.Nifti1Image(recoded_data, bf_image.affine)

# define colormap
colors = ['#2166ac', '#4d84bc', '#79a3cd', '#a6c1dd', '#d2e0ee', '#efd0d4', '#e0a2aa', '#d0747f', '#c14655', '#b2182b']
cmap = ListedColormap(colors)

# load images
bf_ROI = f'{basepath}/BFlog_ROI.nii'
bf_ROI_image = nib.load(bf_ROI)
recoded_ROI_image = recode_bayes_factors(bf_ROI_image)

masked_bf_ROI = f'{basepath}/masked_BFlog_ROI.nii'
masked_bf_ROI_image = nib.load(masked_bf_ROI)
recoded_masked_ROI_image = recode_bayes_factors(masked_bf_ROI_image)

# define slices
def plot_brain_slices(img, title, cmap, save_path):
    coords_display_modes = [
        ([6], 'x'),    # sagittal plane
        ([16], 'y'),   # coronal plane
        ([3], 'z'),    # horizontal plane
        ([33], 'z')    # horizontal plane
    ]
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 18))
    fig.suptitle(title, fontsize=16)
    fig.patch.set_facecolor('white')

    # flatten axes array for easy iteration
    axes = axes.flatten()

    # define subplot positions to reduce whitespace
    positions = [
        [0.1, 0.62, 0.4, 0.3],  # [left, bottom, width, height] for first subplot
        [0.55, 0.62, 0.4, 0.3],  # second subplot
        [0.1, 0.32, 0.4, 0.3],  # third subplot
        [0.55, 0.32, 0.4, 0.3],  # fourth subplot
    ]

    for i, (cut_coords, display_mode) in enumerate(coords_display_modes):
        axes[i].set_position(positions[i])  # set position of subplot
        display = plotting.plot_stat_map(
            img, 
            bg_img=standard_brain,
            cut_coords=cut_coords,
            display_mode=display_mode,
            colorbar=False,
            axes=axes[i],
            draw_cross=False,
            cmap=cmap,
            alpha=0.7,
            black_bg=False)
        display._bg_color = 'white'
        display.frame_axes.set_facecolor('white')
        # font size for axes labels
        axes[i].tick_params(axis='both', which='major', labelsize=14)

    # legend
    categories = [
        'Extreme Evidence for H0', 'Very Strong Evidence for H0', 'Strong Evidence for H0', 'Moderate Evidence for H0',
        'Anecdotal Evidence for H0',  'Anecdotal Evidence for H1', 'Moderate Evidence for H1', 'Strong Evidence for H1', 
        'Very Strong Evidence for H1', 'Extreme Evidence for H1']
    actual_colors = ['#2166ac', '#4d84bc', '#79a3cd', '#a6c1dd', '#d2e0ee', '#efd0d4', '#e0a2aa', '#d0747f', '#c14655', '#b2182b']

    patches = [mpatches.Patch(color=actual_colors[i], label=categories[i]) for i in range(len(categories))]
    legend = fig.legend(handles=patches, title="Evidence Category", loc='center left', bbox_to_anchor=(1.02, 0.5), frameon=False, fontsize=20, title_fontsize=20)

    plt.subplots_adjust(right=0.85)

    # save and show plot
    plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
    plt.show()

# plotting
plot_brain_slices(recoded_ROI_image, 'Bayes Factor ROI Level', cmap, f'{basepath}/BFlog_ROI_image.png')
plot_brain_slices(recoded_masked_ROI_image, 'Bayes Factor ROI Level (Masked for ScanSTRESS Task-Effect)', cmap, f'{basepath}/masked_BFlog_ROI_image.png')
